This Jupyter Notebook has code and examples of how to pull data out of https://perf.skia.org.

For further information see:

  * Pandas: http://pandas.pydata.org/pandas-docs/stable/
  * NumPy:  https://docs.scipy.org/doc/numpy-dev/user/quickstart.html
  * matplotlib: http://matplotlib.org/2.0.0/index.html
  

In [ ]:
# This is where the two functions, perf_calc and perf_query are defined.
#
# See the cells below this one for example of how to use them.
import httplib2
import json
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas


# perf_calc evaluates the formula against the last 50 commits
# and returns a pandas.DataFrame with the results of the calculations.
#
# Example: perf_calc('count(filter(""))')
#
def perf_calc(formula):
    body = {
        'formulas': [formula], 
        'tz': 'America/New_York',
    }
    return perf_impl(body)


# perf_query evaluates the query against the last 50 commits
# and returns a pandas.DataFrame with the results of the query.
#
# Example: perf_query('source_type=skp&sub_result=min_ms')
#
def perf_query(query):
    body = {
        'queries': [query], 
        'tz': 'America/New_York',
    }
    return perf_impl(body)


# utility function.
def noe(x):
    if x == 1e32:
        return np.nan
    else:
        return x

def paramset():
    h = httplib2.Http()
    url = 'https://perf.skia.org/_/initpage/?tz=America/New_York'
    resp, content = h.request(url)
    if resp.status != 200:
        raise "Failed to get initial bounds."
    init = json.loads(content)
    return init['dataframe']['paramset']
    
    
# utility function.
def perf_impl(body):
    h = httplib2.Http()
    url = 'https://perf.skia.org/_/initpage/?tz=America/New_York'
    resp, content = h.request(url)
    if resp.status != 200:
        raise "Failed to get initial bounds."
    init = json.loads(content)
    body['begin'] = init['dataframe']['header'][0]['timestamp']
    body['end'] = init['dataframe']['header'][-1]['timestamp']+1
    (resp, content) = h.request("https://perf.skia.org/_/frame/start", "POST",
                            body=json.dumps(body),
                            headers={'content-type': 'application/json'})
    if resp.status != 200:
        raise "Failed to start query: " + content
    id = json.loads(content)['id']
    state = {'state': 'Starting'}
    url = 'https://perf.skia.org/_/frame/status/' + id
    i = 0
    while state['state'] != 'Success':
        print '\r', '|/-\\'[i%4],
        i+=1
        time.sleep(0.5)
        resp, content = h.request(url)
        if resp.status != 200:
            raise "Failed during query: " + content
        state = json.loads(content)
    url = 'https://perf.skia.org/_/frame/results/' + id
    resp, content = h.request(url)
    if resp.status != 200:
        raise "Failed to load results: " + content

    df = json.loads(content)
    clean = {}
    for key, value in df['dataframe']['traceset'].iteritems():
        clean[key] = [noe(x) for x in value]

    print '\r ',
    return pandas.DataFrame(data=clean)


In [ ]:
# The following line makes the plots interactive.
%matplotlib notebook

# Perform a calculation over Perf data.
df = perf_calc('count(filter(""))')

# pandas.DataFrame's can plot themselves.
df.plot()

In [ ]:
# The following line makes the plots interactive.
%matplotlib notebook

df = perf_query('sub_result=min_ms&test=AndroidCodec_01_original.jpg_SampleSize2_640_480')

# You can also use matplotlib to do the plotting.
plt.plot(df, linestyle='-', marker='o')

In [ ]:
%matplotlib notebook

# DataFrames allow operating on traces in bulk. For example, to 
# normalize each trace to a mean of 0.0 and a std deviation of 1.0:
normed = (df - df.mean())/df.std()
plt.plot(normed)

In [ ]:
%matplotlib notebook
df = perf_query('source_type=skp&sub_result=min_ms')
df.mean(axis=1)

In [ ]:
# Find the noisiest models, from lowest to highest.
#
# Takes a while to run.
params = paramset()
results = pandas.DataFrame()
for model in params['model']:
    df = perf_calc('ave(trace_cov(fill(filter("source_type=svg&sub_result=min_ms&model=%s"))))' % model)
    if df.size > 0:
        df.rename_axis({df.columns[0]: model}, axis="columns")
        results[model] = pandas.Series([df.mean()[0]])
results.sort_values(by=0,axis=1).transpose()

In [ ]:
# Find the differences between CPU and GPU for Chorizo.
#
# Takes a while to run.
params = paramset()
results = pandas.DataFrame()
i = 0
for model in params['name']:
    if model.endswith(".skp") and (model.startswith("top") or model.startswith("key") or model.startswith("desk")):
        print model
        df = perf_calc("""trace_ave(ratio(
  ave(filter("cpu_or_gpu=GPU&model=Chorizo&sub_result=min_ms&name=%s")),
  ave(filter("cpu_or_gpu=CPU&model=Chorizo&sub_result=min_ms&name=%s"))
))""" % (model, model))
        if df.size > 0:
            i+=1
            print model
            if i > 50:
                break
            df.rename_axis({df.columns[0]: model}, axis="columns")
            results[model] = pandas.Series([df.mean()[0]])
results.sort_values(by=0,axis=1).transpose()

In [ ]:
# Find the noisiest models, from lowest to highest.
#
# Takes a while to run.
params = paramset()
results = pandas.DataFrame()
for model in params['test']:
    if model.startswith("GM_"):
        df = perf_calc("""trace_ave(ratio(
  ave(filter("cpu_or_gpu=GPU&model=Chorizo&sub_result=min_ms&test=%s")),
  ave(filter("cpu_or_gpu=CPU&model=Chorizo&sub_result=min_ms&test=%s"))
))""" % (model, model))
        if df.size > 0:
            df.rename_axis({df.columns[0]: model}, axis="columns")
            results[model] = pandas.Series([df.mean()[0]])
results.sort_values(by=0,axis=1).transpose()